### Model Option Building

In [ ]:
import torchvision.models as models
import pandas as pd
import os, sys, torch
sys.path.append('..')

model_types = ['imagenet', 'segmentation', 'detection', 'video']
pytorch_dirs = dict(zip(model_types, ['.', '.segmentation.', '.detection.', '.video.']))

model_archs = {
    'imagenet': ['alexnet','vgg11','vgg13','vgg16','vgg19','vgg11_bn','vgg13_bn','vgg16_bn','vgg19_bn',
                        'resnet18','resnet34','resnet50','resnet101','resnet152',
                        'squeezenet1_0','squeezenet1_1',
                        'densenet121','densenet161','densenet169','densenet201',
                        'googlenet', 'inception_v3',
                        'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'mobilenet_v2',
                        'resnext50_32x4d','resnext101_32x8d','wide_resnet50_2','wide_resnet101_2',
                        'mnasnet0_5','mnasnet1_0',],
    'segmentation': ['fasterrcnn_resnet50_fpn', 'maskrcnn_resnet50_fpn', 'keypointrcnn_resnet50_fpn'],
    'detection': ['deeplabv3_resnet101', 'fcn_resnet101'],
    'video': ['r3d_18', 'r2plus1d_18','mc3_18'],
}

taskonomy_optiomns = {}
for model_type in model_types:
    model_options[model_type] = {}
    
training_calls = {'random': '(pretrained=False)', 'pretrained': '(pretrained=True)'}
    
for model_type, model_list in model_archs.items():
    for model in model_list:
        for training_type in ['random', 'pretrained']:
            model_string = '_'.join([model, training_type])
            model_call = 'models' + pytorch_dirs[model_type] + model + training_calls[training_type]
            model_options[model_type][model_string] = model_call
            
custom_model_options = {
    'alexnet_norelu_random': 'AlexNet_NoRelu()',
    'alexnet_nomaxpool_random': 'AlexNet_NoMaxPool()',
    'alexnet_norelu_nomaxpool_random': 'AlexNet_NoRelu_NoMaxPool()',
    'alexnet_nuconvolve_random': 'AlexNet_NuConvolve()',
}

model_options['custom'] = custom_model_options

In [ ]:
sys.path.append('../toolbox')
import torchvision.transforms as transforms
import video_transforms as video_transforms

model_transforms = {
    'imagenet': transforms.Compose([
        transforms.Resize((224,224)), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225]),
    ]),
    'segmentation': transforms.Compose([
        transforms.Resize((224,224)), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225]),
    ]),
    'detection': transforms.Compose([
        transforms.Resize((224,224)), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225]),
    ]),
    'video': transforms.Compose([
        video_transforms.ToFloatTensorInZeroOne(),
        video_transforms.Resize((224,224)),
        video_transforms.Normalize(mean=[0.43216, 0.394666, 0.37645],
                                   std=[0.22803, 0.22145, 0.216989]),
    ])
}

In [ ]:
model_names = {
    'alexnet': 'AlexNet',
    'vgg11': 'VGG11',
    'vgg13': 'VGG13',
    'vgg16': 'VGG16',
    'vgg19': 'VGG19',
    'vgg11_bn': 'VGG11-BatchNorm',
    'vgg13_bn': 'VGG13-BatchNorm',
    'vgg16_bn': 'VGG16-BatchNorm',
    'vgg19_bn': 'VGG19-BatchNorm',
    'resnet18': 'ResNet18',
    'resnet34': 'ResNet34',
    'resnet50': 'ResNet50',
    'resnet101': 'ResNet101',
    'resnet152': 'ResNet152',
    'squeezenet1_0': 'SqueezeNet1.0',
    'squeezenet1_1': 'SqueezeNet1.1',
    'densenet121': 'DenseNet121',
    'densenet161': 'DenseNet161',
    'densenet169': 'DenseNet169',
    'densenet201': 'DenseNet201',
    'googlenet': 'GoogleNet',
    'shufflenet_v2_x0_5': 'ShuffleNet-V2-x0.5',
    'shufflenet_v2_x1_0': 'ShuffleNet-V2-x1.0',
    'mobilenet_v2': 'MobileNet-V2',
    'resnext50_32x4d': 'ResNext50-32x4D',
    'resnext101_32x8d': 'ResNext50-32x8D',
    'wide_resnet50_2': 'Wide-ResNet50',
    'wide_resnet101_2': 'Wide-ResNet101',
    'mnasnet0_5': 'MNASNet0.5',
    'mnasnet1_0': 'MNASNet1.0',
}

In [ ]:
{k:v for k,v in model_options['detection'].items() 
 if any(s in k for s in model_names['imagenet'])}

In [ ]:
from itertools import compress
list(compress(model_types, [('alexnet' in model_options[key]) for key in list(model_options.keys())]))

In [ ]:
model_info = pd.read_csv('model_information.csv')
pytorch_dirs['inception'] = '.'
model_options = {}

for index, row in model_info.iterrows():
    model = row['model_arch']
    model_type = row['model_type']
    for training_type in ['random', 'pretrained']:
        model_string = '_'.join([model, training_type])
        model_call = 'models' + pytorch_dirs[model_type] + model + training_calls[training_type]
        model_options[model_string] = ({'arch': model, 'type': model_type, 'call': model_call})

In [ ]:
[model_string for model_string in list(model_options.keys()) if model_options[model_string]['type'] == 'imagenet']

### Bash Scripting

In [ ]:
from glob import glob
import os, stat

In [ ]:
with open('compute_model_rdms.sh', 'w') as rsh:
    rsh.write('#! /bin/bash\n')
    for index, model_string in enumerate(model_options.keys()):
        if index < len(model_options.keys()) - 1:
            rsh.write("python compute_model_rdms.py --model_string='{}' & \n".format(model_string))
            rsh.write('wait; \n')
        if index + 1 == len(model_options.keys()):
            rsh.write("python compute_model_rdms.py --model_string='{}'".format(model_string))

In [ ]:
with open('compute_model_rdms.sh', 'w') as rsh:
    rsh.write('#! /bin/bash\n')
    for index, model_string in enumerate(model_options.keys()):
        if index < len(model_options.keys()) - 1:
            rsh.write("python compute_other_model_rdms.py --model_string='{}' & \n".format(model_string))
            rsh.write('wait; \n')
        if index + 1 == len(model_options.keys()):
            rsh.write("python compute_other_model_rdms.py --model_string='{}'".format(model_string))

In [ ]:
with open('mouse_model_rsa.sh', 'w') as rsh:
    rsh.write('#! /bin/bash\n')
    for index, model_string in enumerate(model_options.keys()):
        if index < len(model_options.keys()) - 1:
            rsh.write("python mouse_model_rsa.py --model_string='{}' & \n".format(model_string))
            rsh.write('wait; \n')
        if index + 1 == len(model_options.keys()):
            rsh.write("python mouse_model_rsa.py --model_string='{}'".format(model_string))

In [ ]:
# devices = [0,1,2]
# ppd = [len(array) for array in np.array_split(range(condition_count), len(devices))]
# for device in devices:
#     with open('train_device_{}.sh'.format(device), 'w') as rsh:
#         rsh.write('#! /bin/bash\n')
#         for i in range(ppd[device]):
#             rsh.write('python continue_classifier_training.py --device={} & \n'.format(device))
#             if i < ppd[device] - 1:
#                 rsh.write('wait; \n')
#             if i == ppd[device] - 1:
#                 rsh.write('wait')

### Task Option Building

In [ ]:
import pandas as pd
import torch
import os, sys

In [ ]:
from visual_priors import taskonomy_network

In [ ]:
task_typology = pd.read_csv('task_typology.csv')

In [ ]:
def instantiate_taskonomy_model(model_name):
    weights = torch.load('task_weights/{}_encoder.pth'.format(model_name))
    print('{} weights loaded succesfully.'.format(model_name))
    model = taskonomy_network.TaskonomyEncoder()
    model.load_state_dict(weights['state_dict'])
    
    return model

In [ ]:
def define_taskonomy_options():
    taskonomy_options = {}

    for index, row in task_typology.iterrows():
        model_name = row['model']
        model_type = 'taskonomy'
        train_type = 'taskonomy'
        model_string = model_name
        model_call = "instantiate_taskonomy_model('{}')".format(model_name)
        taskonomy_options[model_string] = ({'model_name': model_name, 'model_type': model_type, 
                                        'train_type': train_type, 'call': model_call})
        
    taskonomy_options['taskonomy_random'] = ({'model_name': 'taskonomy_random', 'model_type': model_type,
                                             'train_type': 'random', 'call': 'taskonomy_network.TaskonomyEncoder()'})
            
    return taskonomy_options

In [ ]:
taskonomy_options = define_taskonomy_options()

In [ ]:
taskonomy_options['autoencoding']

In [ ]:
taskonomy_options['taskonomy_random']

In [ ]:
model1 = eval(taskonomy_options['autoencoding']['call'])

In [ ]:
model2 = eval(taskonomy_options['class_object']['call'])

In [ ]:
def compare_models(model_1, model_2):
    models_differ = 0
    for key_item_1, key_item_2 in zip(model_1.state_dict().items(), model_2.state_dict().items()):
        if torch.equal(key_item_1[1], key_item_2[1]):
            pass
        else:
            models_differ += 1
            if (key_item_1[0] == key_item_2[0]):
                #print('Mismtach found at', key_item_1[0])
                continue
            else:
                raise Exception
    if models_differ == 0:
        print('Models match.')
    if models_differ != 0:
        print('Models differ.')

In [ ]:
compare_models(model1, model2)

### Testing

In [1]:
from model_options import get_model_options

In [5]:
get_model_options(train_type='taskonomy').keys()

dict_keys(['autoencoding', 'class_object', 'class_scene', 'curvature', 'colorization', 'denoising', 'depth_euclidean', 'depth_zbuffer', 'edge_occlusion', 'edge_texture', 'egomotion', 'fixated_pose', 'inpainting', 'jigsaw', 'keypoints2d', 'keypoints3d', 'nonfixated_pose', 'normal', 'point_matching', 'reshading', 'room_layout', 'segment_semantic', 'segment_unsup25d', 'segment_unsup2d', 'vanishing_point'])